In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('hlth_cd_aro.csv')

In [3]:
# Convert all columns to lowercase
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [4]:
df.head()

,dataflow,last_update,freq,unit,sex,age,icd10,resid,geo,time_period,obs_value,obs_flag
0,ESTAT:HLTH_CD_ARO(1.0),14/02/23 23:00:00,Annual,Number,Total,Total,All causes of death,All deaths reported in the country,Austria,2012,78961.0,NaN
1,ESTAT:HLTH_CD_ARO(1.0),14/02/23 23:00:00,Annual,Number,Total,Total,All causes of death,All deaths reported in the country,Austria,2013,79020.0,NaN
2,ESTAT:HLTH_CD_ARO(1.0),14/02/23 23:00:00,Annual,Number,Total,Total,All causes of death,All deaths reported in the country,Austria,2014,77940.0,NaN
3,ESTAT:HLTH_CD_ARO(1.0),14/02/23 23:00:00,Annual,Number,Total,Total,All causes of death,All deaths reported in the country,Austria,2015,83026.0,NaN
4,ESTAT:HLTH_CD_ARO(1.0),14/02/23 23:00:00,Annual,Number,Total,Total,All causes of death,All deaths reported in the country,Austria,2016,80374.0,NaN


In [5]:
df.drop(columns=['dataflow', 'last_update', 'freq', 'unit', 'sex', 'age', 'obs_flag', 'resid'], inplace=True)

In [6]:
# Drop geo too
# df.drop(columns=['geo'], inplace=True)

In [7]:
# Drop 2019 and 2020 as we are using absolute values and Uk is missing after 2018
filter = (df['time_period'] == 2019) | (df['time_period'] == 2020)
df.drop(index=df[filter].index, inplace=True)

In [8]:
# Replace all NaN with 0
df.fillna(0, inplace=True)
df.head()

,icd10,geo,time_period,obs_value
0,All causes of death,Austria,2012,78961.0
1,All causes of death,Austria,2013,79020.0
2,All causes of death,Austria,2014,77940.0
3,All causes of death,Austria,2015,83026.0
4,All causes of death,Austria,2016,80374.0


In [9]:
df.dtypes

icd10           object
geo             object
time_period      int64
obs_value      float64
dtype: object

In [10]:
all = df.groupby(['icd10', 'time_period', 'geo']).sum().reset_index()
all.head()

,icd10,time_period,geo,obs_value
0,Accidental drowning and submersion,2012,Austria,49.0
1,Accidental drowning and submersion,2012,Belgium,57.0
2,Accidental drowning and submersion,2012,Bulgaria,142.0
3,Accidental drowning and submersion,2012,Croatia,135.0
4,Accidental drowning and submersion,2012,Cyprus,18.0


In [11]:
all.rename(columns={"obs_value": "value"}, inplace=True)

In [12]:
all.replace(to_replace='Germany (until 1990 former territory of the FRG)', value='Germany', inplace=True)

In [13]:
all.drop(all[all.geo == 'France (metropolitan)'].index, inplace=True)

In [14]:
top2012 = all[all['time_period'] == 2012].sort_values(by='value', ascending=False).reset_index(drop=True)
top2012_list = top2012['icd10'].tolist()[1:20]

In [15]:
top2012

,icd10,time_period,geo,value
0,All causes of death,2012,Germany,871615.0
1,All causes of death,2012,Italy,613520.0
2,All causes of death,2012,France,571464.0
3,All causes of death,2012,United Kingdom,570691.0
4,All causes of death,2012,Spain,402950.0
...,...,...,...,...
3123,"COVID-19, virus identified",2012,Poland,0.0
3124,"COVID-19, virus identified",2012,Portugal,0.0
3125,"COVID-19, virus identified",2012,Romania,0.0
3126,"COVID-19, virus identified",2012,Serbia,0.0


In [16]:
all.to_csv('death_causes.csv', index=False)